In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale desde s y llega hasta t
beq = np.array([1, 0, 0, 0, 0, -1])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = False)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 e infinito
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

In [3]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([7, 1, 2, 3, 2, 1, 2])
pd.DataFrame(data=[C], columns=nan_names, index=["Costos"])#, columns=nan_names)

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
Costos,7,1,2,3,2,1,2


In [4]:
pd.DataFrame(data=NN, columns=node_names, index=node_names)

,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,0,0,0,0,0,0


In [5]:
pd.DataFrame(data=Aeq, columns=nan_names, index=node_names)

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
s,1,1,0,0,0,0,0
2,-1,0,1,1,0,0,0
3,0,-1,0,0,1,0,0
4,0,0,-1,0,0,1,0
5,0,0,0,0,-1,0,1
t,0,0,0,-1,0,-1,-1


In [6]:
NN_with_cost = convert_cost_to_NxN(C, arc_idxs)
G_with_cost = NN_with_cost.copy()

In [7]:
from IPython.display import display

NN_with_cost = convert_cost_to_NxN(C, arc_idxs)
G_with_cost = NN_with_cost.copy()

it = 0

while True:
    print("Iteración: ", it)
    # Itero una vez:
#     print("NN_With_cost: \n", G_with_cost)

    good_path, visited, stack = dfs_from_NN(G_with_cost)
    
    if good_path == None:
        break
    
    print(visited)
    print(stack)

    residual_G, G_with_cost = min_cost(G_with_cost, good_path)

    # G_with_cost = G_with_cost - residual_G

#     print("NN_With_cost: \n", G_with_cost)
#     print("residual cost: \n", residual_G)
    print("\nG_with_cost")
    display(pd.DataFrame(G_with_cost, index=node_names, columns=node_names).head())
    print("residual_G")
    display(pd.DataFrame(residual_G, index=node_names, columns=node_names).head())
#     print(pd.DataFrame(residual_G, index=node_names, columns=node_names).to_html())
    
    it = it + 1
    
residual_G_final = NN_with_cost - G_with_cost

Iteración:  0
[0. 0. 0. 0. 0. 0.]
[0, 1, 3]
Minimal cost =  1.0

G_with_cost


,s,2,3,4,5,t
s,0.0,6.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,3.0
3,0.0,0.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0


residual_G


,s,2,3,4,5,t
s,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0


Iteración:  1
[0. 0. 0. 1. 0. 0.]
[0, 1]
Minimal cost =  3.0

G_with_cost


...

Iteración:  2
[0. 1. 0. 1. 0. 0.]
[0, 2, 4]
Minimal cost =  1.0

G_with_cost


,s,2,3,4,5,t
s,0.0,3.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0


residual_G


,s,2,3,4,5,t
s,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0


Iteración:  3


In [8]:
print("Se encontraron {} rutas".format(it))

Se encontraron 3 rutas


In [9]:
pd.DataFrame((NN_with_cost!=0).astype(int) - (G_with_cost!=0).astype(int), index=node_names, columns=node_names)

,s,2,3,4,5,t
s,0,0,1,0,0,0
2,0,0,0,0,0,1
3,0,0,0,0,0,0
4,0,0,0,0,0,1
5,0,0,0,0,0,0
t,0,0,0,0,0,0


# Resultados:

In [10]:
idxs = np.argwhere((NN_with_cost!=0).astype(int) - (G_with_cost!=0).astype(int))

In [11]:
critical_nodes = get_col_names((NN_with_cost!=0).astype(int) - (G_with_cost!=0).astype(int), node_names, as_numpy=True, sep = "->")

In [12]:
capacities = get_col_capacites(NN_with_cost, idxs)

In [13]:
print("Los nodos críticos son: ", critical_nodes)

Los nodos críticos son:  ['s->3' '2->t' '4->t']


In [14]:
print("El flujo por los nodos críticos es:")
for i in range(len(critical_nodes)):
    print("Rama: ", critical_nodes[i], " con flujo de ", capacities[i][1])

El flujo por los nodos críticos es:
Rama:  s->3  con flujo de  1.0
Rama:  2->t  con flujo de  3.0
Rama:  4->t  con flujo de  1.0


In [15]:
idxs_final = np.argwhere(residual_G_final)

In [16]:
final_flow_names = get_col_names(residual_G_final, node_names, as_numpy=True, sep = "->")
final_flow = get_col_capacites(residual_G_final, idxs_final)

In [17]:
print("El flujo circulante por cada rama es:")
for i in range(len(final_flow_names)):
    print("Rama: ", final_flow_names[i], " con flujo de ", final_flow[i][1])

El flujo circulante por cada rama es:
Rama:  s->2  con flujo de  4.0
Rama:  s->3  con flujo de  1.0
Rama:  2->4  con flujo de  1.0
Rama:  2->t  con flujo de  3.0
Rama:  3->5  con flujo de  1.0
Rama:  4->t  con flujo de  1.0
Rama:  5->t  con flujo de  1.0
